# Collect data

In [1]:
# import Lib
import pandas as pd
import numpy as np
import urllib.robotparser
import time
from bs4 import BeautifulSoup
import requests
from fractions import Fraction
import re

### Check dữ liệu trên trang này có hợp pháp khi thu thập

In [2]:
rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://store.steampowered.com/robots.txt')
rp.read()

url = 'https://store.steampowered.com/search/?category1=998&ndl=1'
rp.can_fetch('*', url)

True

Dữ liệu hợp pháp!

### Crawl data với các thuộc tính

File steam_game.csv:
- title  
- genre  
- tags  
- category  
- is_dlc  
- is_mature  
- franchise  
- release date  
- developer  
- publisher  
- languages  
- overwhelmingly positive count  
- user reviews count  
- negative count  
- price

File reviews.csv:
- title
- found_helpful
- recommended
- reviews

In [3]:
# Title = []  
# Genre = []  
# Tags = []   
# withDLC = []  
# isMature = [] 
# Franchise = []
# ReleaseDate = []  
# Developer = []  
# Publisher = []
# Languages = []
# PositiveReviews = []
# TotalReviews = []
# NegativeReviews = []  
# OriginalPrice = []
# DiscountPercent = []

In [37]:
data = pd.DataFrame()
data_review = pd.read_csv('reviews.csv')

# data = pd.read_csv('steam_game.csv')
data_review.tail()

,Title,Recommended,Reviews,FoundHelpful
29560,Utawarerumono: Mask of Truth,Recommended,5 Reasons why you should buy this game : 1. Ku...,36 people found this review helpful1 person fo...
29561,Utawarerumono: Mask of Truth,Recommended,TL;DR don't sleep on this title. Despite bein...,22 people found this review helpful\t\t\t\t6
29562,Utawarerumono: Mask of Truth,Recommended,"Don't be alarmed by the playtime, I've finishe...",19 people found this review helpful\t\t\t\t0
29563,Utawarerumono: Mask of Truth,Recommended,"I do not really review games, but I cannot ski...",14 people found this review helpful\t\t\t\t0
29564,Utawarerumono: Mask of Truth,Recommended,Play Utawarerumono,13 people found this review helpful\t\t\t\t0


In [38]:
def crawl_one_game_review(item_review_url, title):
    response = requests.get(item_review_url)
    content = response.text
    if response.status_code == 200:
        soup = BeautifulSoup(content, 'html.parser')
        
        # set all attribute is NaN
        Title = []
        FoundHelpful = []
        Recommended = []
        Reviews = []

        try:
            review_box = soup.find_all('div', attrs={'class': 'apphub_UserReviewCardContent'})

            for i in review_box:
                Title.append(title)

                Recommended.append(i.find('div',attrs={'class':'title'}).string.strip())

                review = i.find('div',attrs={'class':'apphub_CardTextContent'}).text.split('\n')[2].strip()
                # print(review)
                Reviews.append(review)

                found_helpful = i.find('div',attrs={'class':'found_helpful'}).text.split('\n')[1].strip()
                # print(found_helpful)
                FoundHelpful.append(found_helpful)
                
            crawled_data_review = pd.DataFrame({"Title": Title,
                                                 "Recommended": Recommended,
                                                 "Reviews": Reviews,
                                                 "FoundHelpful": FoundHelpful})
            
            global data_review
            data_review = pd.concat([data_review, crawled_data_review], axis = 0)
                
        except:
            pass


In [39]:
def crawl_one_game(item_url):
    response = requests.get(item_url)
    content = response.text
    if response.status_code == 200:
        soup = BeautifulSoup(content, 'html.parser')
        
        # set all attribute is NaN
        title = np.nan 
        genre = np.nan
        tags = np.nan
        with_dlc = np.nan 
        is_mature = np.nan
        franchise = np.nan
        release_date = np.nan
        developer = np.nan
        publisher = np.nan
        languages = np.nan
        positive_reviews = np.nan
        total_reviews = np.nan
        negative_reviews = np.nan

        # Check if a dlc
        is_dlc = soup.find('div',attrs={'class':'game_area_bubble game_area_dlc_bubble'})
        # Check if a soundtrack
        is_soundtrack = soup.find('div',attrs={'class':'game_area_bubble game_area_soundtrack_bubble'})
        
        if is_dlc or is_soundtrack:
            return title, genre, tags, with_dlc, is_mature, franchise, release_date, developer, publisher, languages, positive_reviews, total_reviews, negative_reviews

#         # Check game has DLC
#         try:
#             with_dlc_tmp = soup.find('div',attrs={'id':'gameAreaDLCSection'})
#             with_dlc = (True if with_dlc_tmp else False) 
#         except:
#             pass

        # Title
        try:
            title = soup.find('div',attrs={'class':'apphub_AppName'}).string.strip() 
        except:
            pass

#         # Genere
#         try:
#             genre = [] # A game can have more than one genre
#             genre_tmp = soup.find('div',attrs={'id':'genresAndManufacturer'}).find('span')
#             for i in genre_tmp.find_all('a'):
#                 genre.append(i.text.strip())
                
#             genre = ", ".join(genre)
#         except:
#             genre = np.nan
#             pass

#         # Tag
#         try:
#             tags = [] # A game can have more than one tag
#             for i in soup.find('div',attrs={'class':'glance_tags popular_tags'}).find_all('a'):         
#                 if i.text.strip() != '+':
#                     tags.append(i.text.strip())
                    
#             tags = ", ".join(tags)
#         except:
#             tags = np.nan
#             pass

#         # Check Mature
#         try:
#             is_mature_tmp = soup.find('div',attrs={'id':'game_area_content_descriptors'})
#             is_mature = (True if is_mature_tmp else False)
#         except:
#             pass

#         # Developer, Publisher, Franchise
#         try:
#             for i in soup.find('div',attrs={'id':'genresAndManufacturer'}).find_all('div',attrs={'class':'dev_row'}):
#                 if i.find('b').text == 'Developer:':
#                     developer = i.find('a').text
#                 elif i.find('b').text == 'Publisher:':
#                     publisher = i.find('a').text
#                 elif i.find('b').text == 'Franchise:':
#                     franchise = i.find('a').text
#         except:
#             pass

#         # Release Date
#         try:
#             release_date = soup.find('div',attrs={'class':'date'}).string.strip()
#         except:
#             pass 

#         # Languages
#         try:
#             languages = [] # A game can have more than one language
#             for i in soup.find('div',attrs={'id':'languageTable'}).find_all('td',attrs={'class':'ellipsis'}):   
#                 languages.append(i.text.strip())
            
#             languages = ", ".join(languages)
#         except:
#             languages = np.nan
#             pass

#         # All review
#         try:
#             total_reviews = soup.find('label',attrs={'for':'review_type_all'}).find('span',attrs={'class':'user_reviews_count'}).string.strip()
#             total_reviews = int(total_reviews.replace(')','').replace('(',''))
#         except:
#             pass

#         # Negative review
#         try:
#             negative_reviews = soup.find('label',attrs={'for':'review_type_negative'}).find('span',attrs={'class':'user_reviews_count'}).string.strip()
#             negative_reviews = int(negative_reviews.replace(')','').replace('(',''))
#         except:
#             pass

#         # Positive review
#         try:
#             positive_reviews = soup.find('label',attrs={'for':'review_type_positive'}).find('span',attrs={'class':'user_reviews_count'}).string.strip()
#             positive_reviews = int(positive_reviews.replace(')','').replace('(',''))
#         except:
#             pass
        
        # Crawl all reviews of game
        id_game = re.search(r'\b\d+\b', item_url).group(0)
        url_review = f'https://steamcommunity.com/app/{id_game}/reviews/?browsefilter=toprated'

        crawl_one_game_review(url_review, title)
 
        # Return 
        return title, genre, tags, with_dlc, is_mature, franchise, release_date, developer, publisher, languages, positive_reviews, total_reviews, negative_reviews


In [ ]:
flag_page=True
index=131

# get the games from all the pages and not give it a range because its very dynamic
while(flag_page and index!=201):
#     Title = []  
#     Genre = []  
#     Tags = []   
#     withDLC = []  
#     isMature = [] 
#     Franchise = []
#     ReleaseDate = []  
#     Developer = []  
#     Publisher = []
#     Languages = []
#     PositiveReviews = []
#     TotalReviews = []
#     NegativeReviews = []  
    OriginalPrice = []
    DiscountPercent = []
    print(index)
    
    # Add page parameter
    parameter = '&page='+str(index)
    
    # Send a request to the website
    web_response = requests.get(url+parameter)
        
    # Get the content of the request
    web_content = web_response.text
    
    if web_response.status_code == 200: # request successfully
        web_soup = BeautifulSoup(web_content, 'html.parser')
        
        item_link = []
        
        # Check if the last page
        if index != 1:
            # Find the number of buttons on the page and return False if it has 2
            button = web_soup.find_all('a',attrs={'class':'pagebtn'})
            if len(button)!= 2 :
                flag_page = False
    
        # Add each link to links
        links = web_soup.find('div', attrs={'id': 'search_resultsRows'})
        if links is None:
            index += 1
            continue
    
        for link in links.find_all('a'):
            # Price with discount
            with_discount = link.find('div',attrs={'class':'discount_block search_discount_block'})
            # Price without discount
            without_discount = link.find('div',attrs={'class':'discount_block search_discount_block no_discount'})
            # Free price
            free_price = link.find('div',attrs={'class':'discount_block no_discount search_discount_block'})
            
            if with_discount:
                DiscountPercent.append(with_discount.find('div',attrs={'class':'discount_pct'}).text)
                OriginalPrice.append(with_discount.find('div',attrs={'class':'discount_original_price'}).text)      
            elif without_discount:
                DiscountPercent.append(np.nan)
                OriginalPrice.append(without_discount.find('div',attrs={'class':'discount_final_price'}).text)
            elif free_price:
                DiscountPercent.append(np.nan)
                OriginalPrice.append(free_price.find('div',attrs={'class':'discount_final_price free'}).text)
            else:
                break
        
            item_link.append(link['href'])
            
        # Add attribute of each game
        for item in item_link:    
            title, genre, tags, with_dlc, is_mature, franchise, release_date, developer, publisher, languages, positive_reviews, total_reviews, negative_reviews = crawl_one_game(item)              
            
#             # Add values into attributes
#             Title.append(title)  
#             Genre.append(genre)
#             Tags.append(tags)  
#             withDLC.append(with_dlc)
#             isMature.append(is_mature)
#             Franchise.append(franchise)
#             ReleaseDate.append(release_date) 
#             Developer.append(developer)
#             Publisher.append(publisher)
#             Languages.append(languages)
#             PositiveReviews.append(positive_reviews)
#             TotalReviews.append(total_reviews)
#             NegativeReviews.append(negative_reviews)
#     crawled_data = pd.DataFrame({"Title": Title,
#                      "Genre": Genre,
#                      "Tags": Tags,
#                      "withDLC": withDLC,
#                      "isMature": isMature,
#                      "Franchise": Franchise,
#                      "ReleaseDate": ReleaseDate,
#                      "Developer": Developer,
#                      "Publisher": Publisher,
#                      "Languages": Languages,
#                      "PositiveReviews": PositiveReviews,
#                      "TotalReviews": TotalReviews,
#                      "NegativeReviews": NegativeReviews,
#                      "OriginalPrice": OriginalPrice,
#                      "DiscountPercent": DiscountPercent})   
                     
#     data = pd.concat([data, crawled_data], axis = 0) 
    # Increase page index
    index += 1

131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167


In [33]:
data

""


In [34]:
data_review

,Title,Recommended,Reviews,FoundHelpful
0,Baldur's Gate 3,Recommended,You know that time when you're playing D&D and...,"8,518 people found this review helpful172 peop..."
1,Baldur's Gate 3,Recommended,my work is really cutting into my Baldur's Gat...,"7,638 people found this review helpful2,816 pe..."
2,Baldur's Gate 3,Recommended,Possessed a guard and had them open a gate con...,"3,047 people found this review helpful1,216 pe..."
3,Baldur's Gate 3,Recommended,I have only one point of criticism:This game l...,"1,282 people found this review helpful6 people..."
4,Baldur's Gate 3,Recommended,No microtransactionsNo in-game purchasesNo pay...,"15,312 people found this review helpful49 peop..."
...,...,...,...,...
5,Utawarerumono: Mask of Truth,Recommended,5 Reasons why you should buy this game : 1. Ku...,36 people found this review helpful1 person fo...
6,Utawarerumono: Mask of Truth,Recommended,TL;DR don't sleep on this title. Despite bein...,22 people found this review helpful\t\t\t\t6
7,Utawarerumono: Mask of Truth,Recommended,"Don't be alarmed by the playtime, I've finishe...",19 people found this review helpful\t\t\t\t0
8,Utawarerumono: Mask of Truth,Recommended,"I do not really review games, but I cannot ski...",14 people found this review helpful\t\t\t\t0


In [35]:
# data.to_csv('steam_game.csv', index = False)

In [36]:
data_review.to_csv('reviews.csv', index = False)